In [25]:
import torch
import schnetpack as spk
import math
from schnetpack.datasets import QM9

import numpy as np
from numpy import savetxt

In [26]:
def print_molecule(props):
    
    # load x,y,z coordinates tensors
    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()
    
    for i in range(len(z)):
        if props['_atomic_numbers'][i] == 1:
            print('H',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 6:
            print('C',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 7:
            print('N',x[i],y[i],z[i])     
        if props['_atomic_numbers'][i] == 8:
            print('O',x[i],y[i],z[i])
        if props['_atomic_numbers'][i] == 9:
            print('F',x[i],y[i],z[i]) 
    print('')  
    return z

def hook_v0(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global v0
    #Update the embedding_output variable to be equal to our output tensor
    v0=out_tensor 

def hook_v1(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global v1
    #Update the embedding_output variable to be equal to our output tensor
    v1=out_tensor 

def hook_v2(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global v2
    #Update the embedding_output variable to be equal to our output tensor
    v2=out_tensor  
    
def hook_emb(self, inp_tensor, out_tensor):
    # Self is included and refers to the model class
    # Global allows us to utilize embedding_output outside the current function scope
    global emb
    #Update the embedding_output variable to be equal to our output tensor
    emb=out_tensor 
#def convert_2D(number_of_atoms,rep):
#    layer = np.zeros((number_of_atoms,30))
#    for i in range(number_of_atoms):
#        for j in range(30):
#            layer[i][j] = rep[0][i][j]
#    return layer
def load_checkpoint(qm9data):
    #Load split file 
    train, val, test = spk.data.train_test_split(qm9data,split_file=split_file)

    # Load atom ref data 
    atomrefs = qm9data.get_atomref(QM9.mu)
    # Define SchNet representation model

    schnet = spk.representation.SchNet(
        n_atom_basis=30, n_filters=30, n_gaussians=20, n_interactions=3,
        cutoff=4. , cutoff_network=spk.nn.cutoff.CosineCutoff
    )


    train_loader = spk.AtomsLoader(train, batch_size=100, shuffle=True)
    val_loader = spk.AtomsLoader(val, batch_size=100)
    # Define SchNet output model and property to be predicted
    means, stddevs = train_loader.get_statistics(QM9.mu, divide_by_atoms=True, single_atom_ref=atomrefs)

    output_dip = spk.atomistic.DipoleMoment(n_in=30, property=QM9.mu,
                                   mean=means[QM9.mu], contributions=None, stddev=stddevs[QM9.mu])
    # Define atomistic model

    model = spk.AtomisticModel(representation=schnet,output_modules=output_dip)

    # Load saved checkpoint file
    load_checkpoint = torch.load(checkpoint_path)

    #qm9_i6_30f_20g-1000-500-4_300.pth
    # load model's state dictionary from saved checkpoint
    model.load_state_dict(load_checkpoint)
    
    return model


In [31]:
checkpoint_path = '../../../../data/trained_models/qm9_dipole_trained_1000_500/trained.pth'
split_file ='../../../../data/trained_models/qm9_dipole_trained_1000_500/split.npz'
model_file = "../../../../data/trained_models/qm9_dipole_trained_1000_500/best_model"
number_of_inputs = 5000

# Load QM9 dataset
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

model = load_checkpoint(qm9data)

#set up device for forward pass
device='cpu'

# load atoms converter 
converter = spk.data.AtomsConverter(device=device)

datao = np.zeros((1,30))
datahae = np.zeros((1))
dataoae = np.zeros((1))
datah = np.zeros((1,30))
for idx in range(9,10):
        
    # load data for molecule
    at, props = qm9data.get_properties(idx)
        
    # print molecule for identification
    print(idx)
    z = print_molecule(props)
    number_of_atoms=len(z)
        
    # convert qm9 data to machine-readable form
    inputs = converter(at)
        
    #Instatiate layer output
    v0=None
    v1=None
    v2=None
    x=None       
    
    # Forward hook the model's interaction layer 
    model.representation.interactions[0].register_forward_hook(hook_v0)
        
    # Forward hook the model's interaction layer 
    model.representation.interactions[1].register_forward_hook(hook_v1)
        
    # Forward hook the model's interaction layer 
    model.representation.interactions[2].register_forward_hook(hook_v2)
    
    # Forward hook the model's interaction layer 
    model.representation.embedding.register_forward_hook(hook_emb)
 
#    model = torch.load(model_file, map_location=torch.device('cpu'))
    # Forward pass molecules through the model
    pred = model(inputs)
    
    print('Prediction:', pred[QM9.mu].detach().cpu().numpy()[0,0])
    print('Keys:', list(inputs.keys()))
    print('Truth:', props[QM9.mu].cpu().numpy()[0])

    rep = emb + v0 + v1 + v2
    rows = np.zeros((number_of_atoms,30))
    for i in range(number_of_atoms):
        for j in range(30):
            rows[i][j] = rep[0][i][j]
            
#    from schnetpack.atomistic.output_modules import y
#    from schnetpack.atomistic.output_modules import yi
#    from schnetpack.atomistic.output_modules import result
#    from schnetpack.atomistic.output_modules import charges
    
#    y=y.detach().numpy()
#    
#    print(charges)
#    print(yi)
#    print(y)
#    print(result)
    
    #save the vector of every oxygen atom encountered
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            datao = np.vstack((datao,rows[i]))
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            datah = np.vstack((datah,rows[i]))    
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            datahae = np.vstack((datahae,yi[0][i]))
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 8:
            dataoae = np.vstack((dataoae,yi[0][i]))            



None
9
C -0.017489549 1.2893138 0.0070837247
C 0.0021349336 -0.16746153 -0.00095350837
N 0.018049099 -1.3223433 -0.007233838
H 1.0024261 1.6830851 -0.0005753772
H -0.54359174 1.6669849 -0.873784
H -0.5266811 1.6573676 0.9018339

Prediction: 0.79049164
Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask', 'representation']
Truth: 0.79667646


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
print('DONE')            
savetxt('../../../data/dataO.csv',datao,delimiter=',')
savetxt('../../../data/dataH.csv',datah,delimiter=',')
savetxt('../../../data/hae.csv',datahae,delimiter=',')
savetxt('../../../data/oae.csv',dataoae,delimiter=',')